# THIS IS IMPLEMENTATION OF TRANSFORMER ARCHETECHTURE ON THE EMAIL DETECTION PROJECT

In [1]:
# import libs
import pandas as pd

**1. DATA REFORMATTING**

In [2]:
# load data from final_df.csv
final_df = pd.read_csv('final_df.csv')

# print head
print(final_df.head())
print(final_df.shape)


                                                Text  Result
0                                           forecast       0
1  traveling business meeting take fun trip espec...       0
2                        test test successful way go       0
3  randy send schedule salary level everyone sche...       0
4                            hello let shoot tuesday       0
(519977, 2)


In [3]:
# remove nan values 
final_df = final_df.dropna()

In [4]:
# print shape
print(final_df.shape)

(518920, 2)


**NOW BUILD A CLASS FOR TRANSFORMER**

In [5]:
# import torch
import torch
import torch.nn as nn

In [6]:
class InputEmbedding(nn.Module):
    def __init__(self, embed_size: int, vocab_size: int):
        super().__init__()
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(vocab_size, embed_size)
    
    def forward(self, x):
        return self.embed(x) * torch.sqrt(self.embed_size) # i still don't know why we multiply by sqrt of embed_size 

In [ ]:
class PositionEncoding(nn.Module):

    def __init__(self, embed_size: int = 512, max_len: int = 500, dropout: float):
        super().__init__()
        self.embed_size = embed_size
        self.max_len = max_len
        self.dropout = nn.Dropout(dropout)
        self.posi_embed = nn.Embedding(max_len, embed_size)

        # create a zero matrix 
        pe = torch.zeros(max_len, embed_size)
        # create the pos vector
        pos = torch.arange(0, max_len).unsqueeze(1)
        # calculate the div term
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * -(np.log(10000.0) / embed_size))
        # calculate the pos for even numbers starts from 0
        pe[:, 0::2] = torch.sin(pos * div_term)
        # calculate the pos for odd numbers starts from 1
        pe[:, 1::2] = torch.cos(pos * div_term)

        pe = pe.unsqueeze(0) 

        # save the pe as along with model
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape(1), :]).requires_grad_(False) # fixed

        return self.dropout(x) # prevent overfitting

In [ ]:
class LayerNormalizing(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1)) # change 1 to embed_size if needed
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        norm_x = (x - mean) / (std + self.eps)
        return self.alpha * norm_x  + self.bias

In [ ]:
class FeedForward(nn.Module): # one hidden layer feed forward network
    def __init__(self, embed_size: int, ff_hidden_size: input = 2048, dropout: float):
        super().__init__()
        self.embed_size = embed_size
        self.ff_hidden_size = ff_hidden_size
        self.dropout = nn.Dropout(dropout)

        self.fc1 = nn.Linear(embed_size, ff_hidden_size)
        self.fc2 = nn.Linear(ff_hidden_size, embed_size)

    def forward(self, x):
        return self.fc2(self.dropout(torch.relu(self.fc1(x))))

In [ ]:
class MultiHeadEncoder(nn.Module):
    def __init__(self, embed_size: int, n_heads: int = 128, dropout: float):
        super().__init__()
        self.embed_size = embed_size
        self.n_heads = n_heads
        assert embed_size % n_heads == 0, "Embedding size must be divisible by number of heads"

        self.head_size = embed_size // n_heads
        self.W_q = nn.Linear(embed_size, embed_size)
        self.W_k = nn.Linear(embed_size, embed_size)
        self.W_v = nn.Linear(embed_size, embed_size)

        self.W_o = nn.Linear(embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, Q, K, V, mask):
        Q_p = self.W_q(Q)
        K_p = self.W_k(K)
        V_p = self.W_v(V)
        # slice the Q, K, V to heads
        Q_p = Q_p.view(Q_p.shape[0], Q_p.shape[1], self.n_heads, self.head_size).transpose(1, 2)
        K_p = K_p.view(K_p.shape[0], K_p.shape[1], self.n_heads, self.head_size).transpose(1, 2)
        V_p = V_p.view(V_p.shape[0], V_p.shape[1], self.n_heads, self.head_size).transpose(1, 2)

        # calculate the attention

    @staticmethod
    def attention(Q, K, V, mask, dropout):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(Q.shape[-1], dtype = torch.float32))
        scores = scores.masked_fill(mask == 0, -1e9)
        scores = torch.softmax(scores, dim = -1)
        scores = dropout(scores)
        output = torch.matmul(scores, V)
        return output
